In [1]:
#Import Pandas (for table manipulation), SQLite3 (for the back-end querying), Matplotlib (for charts), and NumPy (for some mathematical operations we may need)
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#Check whether the Redfin data file has changed since last we looked at it; if it has, redownload it, but keep our most recent copy if it has not
import requests
import hashlib
import os

# URL of the Redfin data:
url = "https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_covid19/weekly_housing_market_data_most_recent.tsv000"

# File path in our local directory in which we'll save the data, once a refresh has been made:
local_file_path = "weekly_housing_market_data_most_recent.tsv000"

# Calculate the hash of the file:
def calculate_file_hash(file_path):
    hasher = hashlib.md5()
    with open(file_path, 'rb') as f:
        while True:
            data = f.read(8192)  # Read the file in 8KB chunks
            if not data:
                break
            hasher.update(data)
    return hasher.hexdigest()

# Check if we've already made the local file:
if os.path.exists(local_file_path):
    # Get the hash of the current local file:
    current_hash = calculate_file_hash(local_file_path)

    # Send an HTTP HEAD request to get the file's metadata without downloading it:
    response = requests.head(url)

    # Check if the request was successful (HTTP status code 200):
    if response.status_code == 200:
        # Get the ETag header from the response, if available:
        server_hash = response.headers.get('ETag', '')

        # Compare the local file's hash with the server's ETag:
        if current_hash != server_hash:
            # The file has changed; download and replace it:
            response = requests.get(url)
            if response.status_code == 200:
                with open(local_file_path, 'wb') as local_file:
                    local_file.write(response.content)
                print("File updated.")
            else:
                print(f"Failed to download the updated file. Status code: {response.status_code}")
        else:
            print("File has not changed.")
    else:
        print(f"Failed to check the file. Status code: {response.status_code}")
else:
    # If the local file does not exist, then download it and save it:
    response = requests.get(url)
    if response.status_code == 200:
        with open(local_file_path, 'wb') as local_file:
            local_file.write(response.content)
        print("File downloaded.")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

File updated.


In [210]:
#Import the up-to-date Redfin data into a Pandas dataframe:
df = pd.read_csv("weekly_housing_market_data_most_recent.tsv000", delimiter='\t')

In [211]:
# list(df.columns)

In [212]:
# df.head()

# Select a few columns of interest from the complete list of fields in the Redfin .tsv file:
cols = [
'region_name',
'region_type',
'period_begin',
'period_end',
'average_homes_sold',
'total_homes_sold', 
'total_homes_sold_with_price_drops',
'inventory',
'median_days_on_market',
'median_sale_price',
'price_drops',
'total_new_listings',
'median_new_listing_price'
]

df_slimmed = df[cols]

In [213]:
# Create a connection to the SQLite database
conn = sqlite3.connect('point72.db')

# Write the DataFrame to the database
df_slimmed.to_sql('df_slimmed', conn, if_exists='replace', index=False)

3826582

In [214]:
# Reopen the database connection
conn = sqlite3.connect('point72.db')

# Write a query to calculate total home sold, average sale price, homes sold with price drops, percentage of homes sold with price drop, total inventory, median days on market, and new listings by week:
query = """
select
period_begin
, period_end
, sum(total_homes_sold) total_homes_sold
, sum(median_sale_price*total_homes_sold)/sum(total_homes_sold) avg_sale_price
, sum(total_homes_sold_with_price_drops) homes_sold_with_price_drops
, sum(total_homes_sold_with_price_drops)/sum(total_homes_sold) perc_sold_with_price_drop
, sum(inventory) inventory
, sum(median_days_on_market*total_homes_sold)/sum(total_homes_sold) median_days_on_market
, sum(total_new_listings) total_new_listings
from df_slimmed
where period_end = date(period_begin, '+6 days')
and period_begin >= "2023-01-01"
group by 1,2
order by 1 desc
;
"""
result = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the query result
result

,period_begin,period_end,total_homes_sold,avg_sale_price,homes_sold_with_price_drops,perc_sold_with_price_drop,inventory,median_days_on_market,total_new_listings
0,2023-08-28,2023-09-03,233149.0,403493.130693,60167.0,0.258062,2149136.0,30.822729,216321.0
1,2023-08-21,2023-08-27,204386.0,400308.449400,52200.0,0.255399,2185134.0,30.104021,238937.0
2,2023-08-14,2023-08-20,214089.0,403754.377535,53472.0,0.249765,2182784.0,30.130497,242452.0
3,2023-08-07,2023-08-13,192327.0,402102.930166,46840.0,0.243544,2181660.0,29.317348,243984.0
4,2023-07-31,2023-08-06,232679.0,403015.338905,54837.0,0.235677,2185991.0,29.488903,258263.0
5,2023-07-24,2023-07-30,234514.0,398997.904402,55550.0,0.236873,2199414.0,29.000085,250841.0
6,2023-07-17,2023-07-23,215665.0,399990.499089,50428.0,0.233826,2204338.0,28.763953,260631.0
7,2023-07-10,2023-07-16,220205.0,402783.040757,48738.0,0.221330,2198268.0,27.889512,274651.0
8,2023-07-03,2023-07-09,158413.0,402824.422541,35016.0,0.221042,2178053.0,28.397792,226675.0
9,2023-06-26,2023-07-02,306195.0,407198.087787,67435.0,0.220235,2157798.0,28.291809,255896.0


In [215]:
# Prepare a list of images later to be used in constructing the Excel workbook:
imgs = []

In [216]:
# Sort the DataFrame by 'period_begin' in ascending order
result = result.sort_values(by='period_begin')

# Plot the data
plt.figure(figsize=(12, 6))
plt.plot(result['period_begin'], result['perc_sold_with_price_drop'], marker='x', linestyle='-', label="All")
plt.title('Percent Homes Sold With Price Drop')
plt.xlabel('Week Beginning')
plt.ylabel('Percent Homes Sold With Price Drop')
plt.grid(True)
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

# Show the plot
plt.tight_layout()
plt.legend(loc='upper right')
title = "perc_homes_w_drop.png"
imgs.append(title)
plt.savefig(title)
plt.close()

In [217]:
# Sort the DataFrame by 'period_begin' in ascending order
result = result.sort_values(by='period_begin')

# Plot the data
plt.figure(figsize=(12, 6))
plt.plot(result['period_begin'], result['inventory'], marker='x', linestyle='-', label="All")
plt.title('Total Inventory')
plt.xlabel('Week Beginning')
plt.ylabel('Total Inventory')
plt.grid(True)
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

# Show the plot
plt.tight_layout()
plt.legend(loc='upper right')
title = 'total_weekly_inventory.png'
imgs.append(title)
plt.savefig(title)
plt.close()

In [218]:
# Sort the DataFrame by 'period_begin' in ascending order
result = result.sort_values(by='period_begin')

# Plot the data
plt.figure(figsize=(12, 6))
plt.plot(result['period_begin'], result['median_days_on_market'], marker='x', linestyle='-', label="All")
plt.title('Median Days on Market')
plt.xlabel('Week Beginning')
plt.ylabel('Median Days on Market')
plt.grid(True)
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

# Show the plot
plt.tight_layout()
plt.legend(loc='upper right')
title = 'median_days_on_market.png'
imgs.append(title)
plt.savefig(title)
plt.close()

In [219]:
# Sort the DataFrame by 'period_begin' in ascending order
result = result.sort_values(by='period_begin')

# Plot the data
plt.figure(figsize=(12, 6))
x = result['period_begin']
x1 = [i for i in range(len(x))]
y = result['total_homes_sold']
plt.plot(x, y, marker='x', linestyle='-', label="All: Data")
plt.plot(x, np.poly1d(np.polyfit(x1, y, 1))(x1), label='All: Trendline', linestyle='--')
plt.title('Total Homes Sold')
plt.xlabel('Week Beginning')
plt.ylabel('Total Homes Sold')
plt.grid(True)
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

# Show the plot
plt.tight_layout()
plt.legend(loc='upper right')
title = 'total_homes_sold.png'
imgs.append(title)
plt.savefig(title)
plt.close()

In [220]:
# Sort the DataFrame by 'period_begin' in ascending order
result = result.sort_values(by='period_begin')

# Plot the data
plt.figure(figsize=(12, 6))
x = result['period_begin']
x1 = [i for i in range(len(x))]
y = result['avg_sale_price']
plt.plot(x, y, marker='x', linestyle='-', label="All: Data")
plt.plot(x, np.poly1d(np.polyfit(x1, y, 1))(x1), label='All: Trendline', linestyle='--')
plt.title('Average Sale Price')
plt.xlabel('Week Beginning')
plt.ylabel('Average Sale Price')
plt.grid(True)
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

# Show the plot
plt.tight_layout()
plt.legend(loc='upper right')
title = 'avg_sale_price.png'
imgs.append(title)
plt.savefig(title)
plt.close()

In [221]:
conn = sqlite3.connect('point72.db')

# Execute SQL queries
query = """
with a as (SELECT
period_end
, trim(substr(region_name, instr(region_name, ',') + 1, 3)) st_c
, sum(median_sale_price * total_homes_sold)/sum(total_homes_sold) avg_sale_price
from df_slimmed
where period_end = date(period_begin, '+6 days')
and period_end = "2023-09-03"
group by 1, 2
order by 3 desc
)

select
*
from a
;
"""
state_ranks = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

In [222]:
# Pull home sales, avg prices, etc. by calendar/fiscal quarter:

# Reopen the database connection
conn = sqlite3.connect('point72.db')

# Write a query to calculate total home sold, average sale price, homes sold with price drops, percentage of homes sold with price drop, total inventory, median days on market, and new listings by week:
query = """
select
CASE
WHEN period_begin >= "2023-07-01" then "2023Q3"
WHEN period_begin >= "2023-04-01" then "2023Q2"
WHEN period_begin >= "2023-01-01" then "2023Q1"
WHEN period_begin >= "2022-10-01" then "2022Q4"
WHEN period_begin >= "2022-07-01" then "2022Q3"
WHEN period_begin >= "2022-04-01" then "2022Q2"
WHEN period_begin >= "2022-01-01" then "2022Q1"
WHEN period_begin >= "2021-10-01" then "2021Q4"
WHEN period_begin >= "2021-07-01" then "2021Q3"
WHEN period_begin >= "2021-04-01" then "2021Q2"
WHEN period_begin >= "2021-01-01" then "2021Q1"
WHEN period_begin >= "2020-10-01" then "2020Q4"
WHEN period_begin >= "2020-07-01" then "2020Q3"
WHEN period_begin >= "2020-04-01" then "2020Q2"
ELSE "error"
END as quarter
, sum(total_homes_sold) total_homes_sold
, sum(median_sale_price*total_homes_sold)/sum(total_homes_sold) avg_sale_price
, sum(total_homes_sold_with_price_drops) homes_sold_with_price_drops
, sum(total_homes_sold_with_price_drops)/sum(total_homes_sold) perc_sold_with_price_drop
, sum(inventory) inventory
, sum(median_days_on_market*total_homes_sold)/sum(total_homes_sold) median_days_on_market
, sum(total_new_listings) total_new_listings
from df_slimmed
where period_begin >= "2020-04-01"
and period_end = date(period_begin, '+6 days')
group by 1
order by 1 desc
;
"""

results_by_qtr = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the query result
# result.to_clipboard(index=False, header=True)
results_by_qtr

,quarter,total_homes_sold,avg_sale_price,homes_sold_with_price_drops,perc_sold_with_price_drop,inventory,median_days_on_market,total_new_listings
0,2023Q3,1905427.0,401894.850953,457248.0,0.239971,19664778.0,29.355098,2212755.0
1,2023Q2,2901490.0,397456.334596,698959.0,0.240897,27342638.0,30.853405,3491977.0
2,2023Q1,2149707.0,371378.463110,738538.0,0.343553,26896571.0,47.580410,2780364.0
3,2022Q4,2471258.0,372179.269431,811206.0,0.328256,32231044.0,38.897886,2332044.0
4,2022Q3,3213838.0,385118.600996,716784.0,0.223031,34271296.0,32.283337,3717949.0
5,2022Q2,3614371.0,407434.126206,385097.0,0.106546,26870785.0,22.420696,4595505.0
6,2022Q1,2930275.0,378559.393149,416242.0,0.142049,20815548.0,31.035448,3452276.0
7,2021Q4,3565476.0,365824.554029,614343.0,0.172303,26342475.0,30.540005,2901502.0
8,2021Q3,3961231.0,363870.793886,489604.0,0.123599,30575331.0,25.880774,4294418.0
9,2021Q2,4014133.0,361048.798291,393623.0,0.098059,27122590.0,25.705201,4577942.0


In [223]:
# Look at historical Redfin earnings:

earnings = pd.read_csv("redfin_earnings.csv")

#Join to "results_by_qtr" as above
earnings_join_results = pd.merge(results_by_qtr, earnings, left_on='quarter', right_on='qtr_beginning', how='inner')

earnings_join_results

,quarter,total_homes_sold,avg_sale_price,homes_sold_with_price_drops,perc_sold_with_price_drop,inventory,median_days_on_market,total_new_listings,qtr_beginning,revenue
0,2023Q2,2901490.0,397456.334596,698959.0,0.240897,27342638.0,30.853405,3491977.0,2023Q2,275.6
1,2023Q1,2149707.0,371378.463110,738538.0,0.343553,26896571.0,47.580410,2780364.0,2023Q1,325.7
2,2022Q4,2471258.0,372179.269431,811206.0,0.328256,32231044.0,38.897886,2332044.0,2022Q4,479.7
3,2022Q3,3213838.0,385118.600996,716784.0,0.223031,34271296.0,32.283337,3717949.0,2022Q3,600.5
4,2022Q2,3614371.0,407434.126206,385097.0,0.106546,26870785.0,22.420696,4595505.0,2022Q2,606.9
5,2022Q1,2930275.0,378559.393149,416242.0,0.142049,20815548.0,31.035448,3452276.0,2022Q1,597.3
6,2021Q4,3565476.0,365824.554029,614343.0,0.172303,26342475.0,30.540005,2901502.0,2021Q4,643.1
7,2021Q3,3961231.0,363870.793886,489604.0,0.123599,30575331.0,25.880774,4294418.0,2021Q3,540.0
8,2021Q2,4014133.0,361048.798291,393623.0,0.098059,27122590.0,25.705201,4577942.0,2021Q2,471.0
9,2021Q1,3049596.0,330714.324977,490964.0,0.160993,25238806.0,38.759965,3507792.0,2021Q1,268.0


In [224]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X = earnings_join_results.drop("revenue", axis=1)
X = X.drop("quarter",axis=1)
X = X.drop("qtr_beginning",axis=1)

Y = earnings_join_results["revenue"]

model = LinearRegression()
model.fit(X, Y)

y_pred = model.predict(X)

mse = mean_squared_error(Y, y_pred)
r2 = r2_score(Y, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

test_X = results_by_qtr[results_by_qtr["quarter"] == "2023Q3"]
test_X = test_X.drop("quarter", axis = 1)
scale_factor = 90/63
test_X["total_homes_sold"] *= scale_factor
test_X["homes_sold_with_price_drops"]*= scale_factor
test_X["inventory"]*= scale_factor
test_X["total_new_listings"]*= scale_factor

final_pred = model.predict(test_X)
final_pred

Mean Squared Error: 6521.0680190022495
R-squared: 0.7421050013725532


array([528.66409586])

In [225]:
#Assemble Excel workbook:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows

# Create a new Excel workbook
wb = openpyxl.Workbook()

# Create a sheet called "README"
readme_sheet = wb.create_sheet(title="README")

# Merge cells A1 to E5
readme_sheet.merge_cells('A1:E5')

# Set the text to Cell A1
readme_cell = readme_sheet['A1']
readme_cell.value = "The following tabs contain several charts which, if the preceding cells are re-run, will be updated on an ongoing basis. These charts contain trended data on several indicators key to Redfin's 2023 performance, among them information on home sale quantities, average sale prices, available inventory, waiting time to market, and other useful metrics."

# Enable text wrapping
readme_cell.alignment = openpyxl.styles.Alignment(wrap_text=True)

# Paste the underlying SQL table to the workbook:
df = result
sheet = wb.create_sheet(title="Table")
for row in dataframe_to_rows(df, index=False, header=True):
    sheet.append(row)

# Paste the state ranks table to the workbook:
df = state_ranks
sheet = wb.create_sheet(title="State Ranks")
for row in dataframe_to_rows(df, index=False, header=True):
    sheet.append(row)

# Paste charts as images to individual tabs in the workbook:
for name in imgs:
    sheet = wb.create_sheet(title=name.replace(".png",""))  # Create a new sheet/tab
    chart_image = Image(name)  # Load the saved chart image
    chart_image.anchor = 'A1'  # Set the image position in the sheet
    sheet.add_image(chart_image)  # Add the image to the sheet

#Add in table used for regressing historical quarterly revenue on drivers:
df = earnings_join_results
sheet = wb.create_sheet(title="Revenue Prediction")
for row in dataframe_to_rows(df, index=False, header=True):
    sheet.append(row)

# Clean up the resulting workbook:
if 'Sheet' in wb.sheetnames:
    # Get the "Sheet" sheet
    sheet = wb['Sheet']
    # Delete the sheet
    wb.remove(sheet)

# Save the workbook:
wb.save('Point72.xlsx')